In [1]:
import numpy as np
import pandas as pd
import math
from numba import jit, njit
from tqdm import tqdm

from scipy import signal
import os, sys, time

In [2]:
def calc_ACF(array_1D):
    
    #-------------Normalization
    #yunbiased = array_1D - np.mean(array_1D, axis=0)
    #ynorm = np.sum(np.power(yunbiased,2), axis=0)

    autocor = signal.fftconvolve(array_1D,
                                 array_1D[::-1],
                                 mode='full')[len(array_1D)-1:]   # / ynorm
    return autocor

In [13]:
#--------------------Parameters-----------------------------------
file_path = '.'
file_name = 'AgI-MD-pos-750K.xyz'
nframes = 80000
delta_t = 0.5
numAg = 54
numI = 54
conv= 0.529177 # in Angstrom
box = np.array([0.2883061E+02*conv, 0.2883061E+02*conv, 0.2883061E+02*conv]) # in Angstrom
outputf = 'test.dat'

In [4]:
def read_xyz_file(filename):
    datatypes = {"atoms" : "str", "x" : np.float64, "y" : np.float64, "z" : np.float64}
    dataframe = pd.read_csv(filename, usecols=[0,1,2,3], delim_whitespace=True, names=['atoms','x','y','z'], dtype=datatypes, on_bad_lines='skip', na_values=['='], comment='#').dropna()  
    #dataframe = pd.read_csv(filename, usecols=[0,1,2,3], delim_whitespace=True, names=['atoms','x','y','z']).dropna()   #, dtype=datatypes
    cation_rows = dataframe[dataframe['atoms'] == 'Ag']
    anion_rows = dataframe[dataframe['atoms'] == 'I']
    cation_np = cation_rows[['x', 'y', 'z']].to_numpy()
    anion_np = anion_rows[['x', 'y', 'z']].to_numpy()
    cations = np.reshape(cation_np, (nframes, numAg, 3))
    anions = np.reshape(anion_np, (nframes, numI, 3))
    return cations, anions

In [5]:
def interatomic_separation(typeA, typeB, box):
    # Calculate the squared differences between the x, y, and z coordinates
    squared_differences = ((typeA[:, :, np.newaxis, :] - typeB[:, np.newaxis, :, :]) - np.rint((typeA[:, :, np.newaxis, :] - typeB[:, np.newaxis, :, :])/box)*box )** 2
    
    # Sum the squared differences along the last axis to get the squared distance
    squared_distance = np.sum(squared_differences, axis=-1)
    
    # Take the square root to get the Euclidean distance
    distance = np.sqrt(squared_distance)
    
    return distance 
    

In [6]:
def neighbour_list_changes(distance, numA, nframes, threshold = 3.6):
    neighbour_list = {}
    
    h_list = np.zeros((nframes, numA))
    for i in tqdm(range(nframes)):
        for j in range(numA):
            if (i == 0):
                mask = distance[i,j,:] < threshold
                neighbour_list[(i,j)] = np.where(mask)
            else:
                mask = distance[i,j,:] < threshold
                neighbour_list[(i,j)] = np.where(mask)
                h_list[i,j] = compare_arrays(neighbour_list[(i,j)],neighbour_list[(i-1,j)])
    return h_list

In [7]:
def make_neighbour_list(distance, numA, nframes, threshold = 3.6):
    h_list = np.zeros((nframes, numA, 10))

    for i in range(numA):
        for j in tqdm(range(nframes)):
            mask = distance[j,i,:] < threshold
            neighbours = np.where(mask)[0]
            h_list[j,i,:len(neighbours)] = neighbours
    return h_list

In [8]:
def compare_dict(dic1, dic2):
    if dic1 == dic2:
        return 1
    else:
        return 0

In [9]:
def compare_arrays(arr1, arr2):
    if np.array_equal(arr1, arr2):
        return 1
    else:
        return 0

def compare_arrays2(a,b):
    if a.shape!=b.shape:
        return 0
    else:
        return bool(np.all(a == b))

In [10]:
def save_results(fout, time, intensity):
    with open(fout, "w") as fw:
        title = ("#Time", "R_ACF", "ps", "a.u.")
        np.savetxt(fout, np.c_[time,intensity],
                   fmt="%10.5f %15.5e",
                   header="{0:>10}{1:>16}\n{2:^11}{3:^20}".format(*title),
                   comments='')

In [11]:
def main(file_path, file_name):
    """ The mian workflow:
    (1) read the coordinates of the selected atoms from file;
    (2) calculate the distance between cations and anions;
    (3) Make a neighbour list of all anions around cations;
    (4) Make a residence list (which is the instances when solvation shell of any cation changes);
    (5) Calculate the residence time autocorrelation function of the selected atoms;
    (6) save the result to a txt file
    """
    fname = os.path.join(file_path, file_name)
    
    print("Data-reading Started:")
    start = time.process_time()
    
    cations, anions = read_xyz_file(fname)

    read_data_time = time.process_time() - start

    print("\n".join(("\nData-reading Completed.",
                     "Used time: {:.5f} second.",
                     "The traj. has {:d} time-steps")).format(read_data_time,
                                                              len(cations)))
 
    parsing = time.process_time()

    distance = interatomic_separation(cations, anions, box)
    data = make_neighbour_list(distance, numAg, nframes)

    
    finish = time.process_time()
    print("Job Completed! Used time: {:.5f} second.".format(finish - parsing))
    
    return data


In [14]:
######## The main program ########
if __name__ == "__main__":
    n_list = main(file_path, file_name)

Data-reading Started:

Data-reading Completed.
Used time: 6.43362 second.
The traj. has 80000 time-steps


100%|██████████████████████████████████████████████████████████| 80000/80000 [00:00<00:00, 306449.68it/s]


Job Completed! Used time: 44.15739 second.


In [15]:
new_n_list = n_list.reshape(-1,n_list.shape[-1])

In [16]:
new_n_list.astype(int)

array([[ 4, 14, 15, ...,  0,  0,  0],
       [13, 19, 20, ...,  0,  0,  0],
       [31, 35, 48, ...,  0,  0,  0],
       ...,
       [11, 42, 43, ...,  0,  0,  0],
       [21, 40, 46, ...,  0,  0,  0],
       [ 0, 19, 37, ...,  0,  0,  0]])

In [17]:
np.savetxt('complete_neighbour_list.dat', new_n_list, fmt='%d')